# Классификация

[Ссылка на соревнование](https://www.kaggle.com/c/changellenge-cupit-2019-posneg)

In [0]:
import numpy as np
import pandas as pd

import re
from nltk.corpus import stopwords
import nltk
import string

import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC

In [0]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Загрузка данных

In [0]:
!mkdir ~/.kaggle
!mv kaggle.json /root/.kaggle
!kaggle competitions download -c changellenge-cupit-2019-type

mkdir: cannot create directory ‘/root/.kaggle’: File exists


## Подготовка данных

In [0]:
train_df = pd.read_csv('train_data.csv.zip', index_col=0)
train_df.head()

FileNotFoundError: ignored

In [0]:
train_df['data'] = train_df['title'] + ' ' + train_df['text']
train_df.drop(['title', 'text'], axis=1, inplace=True)
train_df.rename(index=str, columns={"data": "text"}, inplace=True)

In [0]:
set(train_df['type'])

In [0]:
train_df['type'] = train_df['type'].map(
    {
        'Бизнес услуги': 0,
        'Вклад': 1,
        'Дебетовая карта': 2,
        'Денежные переводы': 3,
        'Инвестиционные продукты': 4,
        'Ипотека': 5,
        'Кредит': 6,
        'Обслуживание физ. и юр. лиц': 7,
        'Реструктуризация': 8
    })

In [0]:
train_df.head()

Преобразуем данные в отдельные списки для удобства дальнейшего анализа

In [0]:
target = train_df['type']
text = train_df['text']

In [0]:
text[0]

'Просьба об отмене штрафа КАСКО Здравствуйте! 10.06.2016г. мною был заключен договор №С04101713432 с ООО «Сетелем Банк» о предоставлении целевого потребительского кредита на приобретение автотранспортного средства марки Ford сроком на 3 (три) года. По настоящее время кредитный договор является действующим, финансовые обязательства по погашению задолженности мною выполняются добросовестно. В кредитный договор включен пункт об обязанности страхования автомобиля по КАСКО.В силу ст. 972 ГК РФ страхование может носить обязательный характер только в случаях, предусмотренных законом. Страхование же транспортного средства по полису КАСКО в силу закона носит добровольный характер. Обязанность заемщика страховать транспортное средство по полису КАСКО законом не предусмотрена. В силу п. 2 статьи 16 Закона РФ от 07.02.1992 № 2300-1 (в ред. От 13.07.2015г.) «О защите прав потребителей» запрещается обуславливать приобретение одних товаров (работ, услуг) обязательным приобретением иных товаров (работ

Чистим текст

In [0]:
def tokenize_me(file_text):
    #firstly let's apply nltk tokenization
    tokens = nltk.word_tokenize(file_text)
 
    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]
 
    #deleting stop_words
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'п', 'рф', 'ред', 'г', 'ооо', 'ст', 'гк', 'здравствуйте', 'из-за', '–'])
    tokens = [i for i in tokens if ( i not in stop_words )]
 
    #cleaning words
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    tokens = [i.replace("“", "").replace("”", "") for i in tokens]
    tokens = [i.replace("`", "") for i in tokens]
    tokens = [i.replace("'", "") for i in tokens]
    tokens = [i.replace("ё", "е") for i in tokens]
 
    return tokens

In [0]:
def text_cleaner(text):
  clean = []
  for s in text:
      t = ""
      t = re.sub('(E-mail\S+)', "", s)
      t = re.sub('(@\S+)', "", t)
      t = re.sub('(№\S+)', "", t)
      t = re.sub('(№)', "", t)
      t = re.sub('(\*\S+)', "", t)
      t = re.sub('\.', " ", t)
      t = re.sub('  ', " ", t)
      t = re.sub('(#\w+)|(@\w+)|(\d+)|(&gt;)|(&lt;)', "", t)

      t = t.lower()


      s = ""
      for w in tokenize_me(t):
          s = s + ' ' + w

      t = s
      t = re.sub('  ', " ", t)

      if (t[0]==' '):
          t = t[1:]

      clean.append(t)
  return clean

In [0]:
clean = text_cleaner(text)
clean[0]

'просьба отмене штрафа каско мною заключен договор сетелем банк предоставлении целевого потребительского кредита приобретение автотранспортного средства марки ford сроком года настоящее время кредитный договор является действующим финансовые обязательства погашению задолженности мною выполняются добросовестно кредитный договор включен пункт обязанности страхования автомобиля каско силу страхование носить обязательный характер случаях предусмотренных законом страхование транспортного средства полису каско силу закона носит добровольный характер обязанность заемщика страховать транспортное средство полису каско законом предусмотрена силу статьи закона защите прав потребителей запрещается обуславливать приобретение одних товаров работ услуг обязательным приобретением иных товаров работ услуг понимаю дополнительным способом обеспечения возвратности кредита является полис каско года исправно исполнял обязанность пролонгации полиса каско исходя вышеуказанного истечении срока действия полиса 

## TFIDF Bag of Words

In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=None)
tfidf_vectorizer.fit(clean)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [0]:
x_tfidf = tfidf_vectorizer.transform(clean)
tfidf

<9151x542785 sparse matrix of type '<class 'numpy.float64'>'
	with 1367172 stored elements in Compressed Sparse Row format>

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf, target, test_size=0.2)

## Создание и обучение модели

In [0]:
model = LinearSVC(C=1).fit(x_train, y_train)

In [0]:
predicted = model.predict(x_test)
metrics.accuracy_score(y_test, predicted)

0.6646641179683234

In [0]:
kfold = sklearn.model_selection.KFold(n_splits=10, random_state=7)

scoring = 'accuracy'
accuracy = sklearn.model_selection.cross_val_score(model, 
                tfidf, target, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: ", (accuracy.mean()))

10-fold cross validation average accuracy:  0.6807974801345836


## Тренировка на всех данных

In [0]:
test_df = pd.read_csv('test_data.csv.zip', index_col=0)
test_df['data'] = test_df['title'] + ' ' + test_df['text']
test_df.drop(['title', 'text'], axis=1, inplace=True)
test_df.rename(index=str, columns={"data": "text"}, inplace=True)
test_df.head()

,text
0,Благодарность менеджеру по продажам Хотелось б...
1,Брал рассрочку в магазине Самсунг Здравствуйте...
2,Тинькофф оказался лучшим Работаем с ними уже б...
3,Недавно открыла расчетный счет в Тинькофф Банк...
4,Волею судеб у меня появилась кредитная карта С...


In [0]:
text_t = test_df['text']
clean_t = text_cleaner(text_t)
clean_t[0]

In [0]:
model = LinearSVC(C=1).fit(x_tfidf, target)

In [0]:
test_tfidf = tfidf_vectorizer.transform(clean_t)
test_tfidf

<2750x542785 sparse matrix of type '<class 'numpy.float64'>'
	with 267448 stored elements in Compressed Sparse Row format>

In [0]:
predicted = model.predict(test_tfidf)

In [0]:
categories = ['Бизнес услуги',
 'Вклад',
 'Дебетовая карта',
 'Денежные переводы',
 'Инвестиционные продукты',
 'Ипотека',
 'Кредит',
 'Обслуживание физ. и юр. лиц',
 'Реструктуризация']

In [0]:
categories[predicted[5]]

'Ипотека'

In [0]:
test_df.iloc[5]['text']

'Оформление ипотеки Добрый день! Оформили ипотеку в Сбербанке России,очень довольны работой менеджера по ипотечному кредитованию Ш-вой Ралины отделение 8598/0195. Менеджер узнавала звонила интересовалась, консультировала по телефону во время поиска квартиры. Так же очень помогла в консультации вопросов которые встречались по ходу сделки. Приятно работать с такими хорошими сотрудниками! Спасибо большое! Побольше бы таких\xa0добрых, умных, пунктуальных специалистов!!!'

In [0]:
p = [categories[predicted[i]] for i in range(predicted.shape[0])]

In [0]:
prediction = pd.DataFrame()
prediction['index'] =  [i for i in list(test_df.index)]
prediction['type'] = p

In [0]:
prediction.head(10)

,index,type
0,0,Ипотека
1,1,Кредит
2,2,Обслуживание физ. и юр. лиц
3,3,Обслуживание физ. и юр. лиц
4,4,Кредит
5,5,Ипотека
6,6,Вклад
7,7,Кредит
8,8,Ипотека
9,9,Кредит


In [0]:
# write finilased results into csv
prediction.to_csv('prediction_class.csv', index = False, sep = ',')